In [ ]:
import jsonlines, json
from nltk.stem import PorterStemmer

path = ["data/inverted_index_1.jsonl", "data/inverted_index_2.jsonl"]
inverted_index = {}
url_map = {}

In [ ]:
# read the jsonl file to save the inverted index to a map
with jsonlines.open(path[0], 'r') as file:
    for jsonObj in file:
        token = list(jsonObj.keys())[0]
        inverted_index[token] = jsonObj[token]

In [ ]:
# read the jsonl files to save the inverted index to a map
with jsonlines.open(path[1], 'r') as file:
    for jsonObj in file:
        token = list(jsonObj.keys())[0]
        inverted_index[token] = jsonObj[token]

In [ ]:
# for debugging purpose
# i = 0
# for key in inverted_index:
#     print(key, inverted_index[key])
#     i += 1

#     if i > 10: break

In [ ]:
# read the json file to save the urls' id to a map
with open("url_id.json", 'r') as file:
    data = json.load(file)
    for id in data:
        url_map[id] = data[id]

In [ ]:
# for debugging purpose 
# i = 0
# for key in url_map:
#     print(key, url_map[key])
#     i += 1

#     if i > 10: break

In [ ]:
while True:

    posting_lists = []      # for saving all the url id 
    searching_tokens = [] 
    
    # ask for input/quries
    queries = input("Input queries (type exit to stop searching): ")

    if queries.lower() == "exit":
        break

    # perform the search
    # cristina lopes, machine learning, ACM, master of software engineering

    searching_tokens = queries.split()  # ex: "machine learning" => ['machine', 'learning']

    # note: need a stemming step here before start searching

    stemmer = PorterStemmer()
    stemmed_searching_tokens = [stemmer.stem(word) for word in searching_tokens]

    # get the posting list from each token and save to posting_lists array
    for token in stemmed_searching_tokens:
        if token in inverted_index:
            posting_lists.append(inverted_index[token])
        else:
            print(token, "is not found")

    # find the intersection of the posting lists
    url_id_set = set()
    for posting_list in posting_lists:
        if len(url_id_set) == 0:
            url_id_set.update(set(posting_list.keys()))
        else:
            # url_id_set.intersection(set(posting_list.keys()))
            url_id_set &= set(posting_list.keys())

    # output the urls
    i = 0
    print("URLs that contain", queries)
    for id in url_id_set:
        if i > 5: break
        print(url_map[id])
        i += 1
    print()


In [ ]:
!pip install nltk jsonlines

from nltk.stem import PorterStemmer
import jsonlines, json
import math

N = 55393 # total number of documents
stemmer = PorterStemmer()

# Load inverted index
path = ["data/inverted_index_1.jsonl", "data/inverted_index_2.jsonl"]
inverted_index = {}

for p in path:
    with jsonlines.open(p, 'r') as file:
        for jsonObj in file:
            token = list(jsonObj.keys())[0]
            inverted_index[token] = jsonObj[token]

# Load URL map
url_map = {}
with open("url_id.json", 'r') as file:
    data = json.load(file)
    for id in data:
        url_map[id] = data[id]

# Query loop
while True:
    queries = input("Input queries (type exit to stop searching): ")
    if queries.lower() == "exit":
        break

    tokens = queries.split()
    stemmed_tokens = [stemmer.stem(t) for t in tokens]

    posting_lists = []
    for token in stemmed_tokens:
        if token in inverted_index:
            posting_lists.append(inverted_index[token])
        else:
            print(token, "is not found")

    # Boolean AND: intersect all posting lists
    url_id_set = set()
    for posting_list in posting_lists:
        doc_ids = set(posting_list.keys())
        if not url_id_set:
            url_id_set = doc_ids
        else:
            url_id_set &= doc_ids

    # compute tf-idf scores
    doc_scores = {}

    for doc_id in url_id_set:
        score = 0.0
        for token in stemmed_tokens:
            posting = inverted_index.get(token, {})
            tf = posting.get(doc_id, 0)
            df = len(posting) if posting else 1
            idf = math.log(N / (1 + df))
            score += tf * idf
        doc_scores[doc_id] = score

    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

    # Output up to 5 URLs (no ranking yet)
    print(f"\nURLs that contain: \"{queries}\"")
    for i, (doc_id, score) in enumerate(ranked_docs[:5]):
        url = url_map.get(doc_id, doc_id)
        print(f"{i+1}. {url} (tf-idf: {score:.4f})")
    print()
